In [18]:
import numpy as np 
import pandas as pd 

def objective_function(x, test, prob):
    a, b, c = x
    return np.sum((a * test[0] + b * test[1] + c * test[2] - prob) ** 2)

def constraint(x):
    return np.sum(x) - 1

# create a random np vector of probabilities
prob = np.random.random(500)

test = np.random.random((3,500))

x0 = np.array([1/3, 1/3, 1/3])

In [19]:
objective_function(x0, test, prob)

53.964975079416796

In [20]:
import numpy as np
from scipy.optimize import minimize

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1

result = minimize(objective_function, x0, args=(test, prob), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

Optimal values:
a = 0.3520261659868651
b = 0.33936192866185355
c = 0.30861190535128136
Objective function value: 53.92316611331649


Hamming Distance

In [56]:
import os
import pandas as pd
import numpy as np

path = '/data1/malto/shroom'

results1 = pd.read_csv(path + '/baseline_val.csv')
results2 = pd.read_csv(path + '/crlft_val.csv')
results3 = pd.read_csv(path + '/sequential_nogpt_aware.csv')

In [71]:
r1 = 1 - results1['baseline'].to_numpy()
r2 = results2['deberta-xlarge-mnli_0.01_frozen_22_24_1_0.772_63_sequential'].to_numpy()
r3 = 1 - results3['sequential_nogpt_deberta_xlarge'].to_numpy()

test = np.stack((r1, r2, r3), axis=0).mean(axis=0)
test.shape

(501,)

In [73]:
labels = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
labels.shape

(501,)

In [74]:
predictions = np.where(test > 0.5, 1, 0)
labels = np.where(labels > 0.5, 1, 0)
average_wrongs = np.logical_xor(predictions, labels).sum()
average_wrongs

113

In [75]:
labels1 = labels[:len(labels)//2]
labels2 = labels[len(labels)//2:]

r1a = r1[:len(r1)//2]
r1b = r1[len(r1)//2:]

r2a = r2[:len(r2)//2]
r2b = r2[len(r2)//2:]

r3a = r3[:len(r3)//2]
r3b = r3[len(r3)//2:]

test1 = np.stack((r1a, r2a, r3a), axis=0)
test2 = np.stack((r1b, r2b, r3b), axis=0)

In [79]:
import numpy as np 
import pandas as pd 

def objective_function(x, test, prob):
    a, b, c = x
    return np.sum((a * test[0] + b * test[1] + c * test[2] - prob) ** 2)

def constraint(x):
    return np.sum(x) - 1

# create a random np vector of probabilities
prob = labels1 

test = np.stack((r1, r2, r3), axis=0)

x0 = np.array([1/3, 1/3, 1/3])

In [80]:
objective_function(x0, test1, labels1), objective_function(x0, test2, labels2)

(39.355011407649656, 37.52143337508387)

In [85]:
import numpy as np
from scipy.optimize import minimize

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1

result = minimize(objective_function, x0, args=(test, labels), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

Optimal values:
a = 0.0
b = 0.32036973827272186
c = 0.6796302617274056
Objective function value: 71.19941092702837


In [84]:
objective_function(result.x, test2, labels2)

35.860131829091316

In [86]:
import os
import pandas as pd
import numpy as np

path = '/data1/malto/shroom'

results1 = pd.read_csv(path + '/baseline.csv')
results2 = pd.read_csv(path + '/crlft.csv')
results3 = pd.read_csv(path + '/sequential_nogpt.csv')

In [88]:
results2

,crlft_alldata_deberta_xlarge_forzen_22_sequential
0,0.045296
1,0.128770
2,0.012510
3,0.928744
4,0.237917
...,...
1495,0.013449
1496,0.923321
1497,0.012414
1498,0.013565


In [89]:
r1 = 1 - results1['baseline'].to_numpy()
r2 = results2['crlft_alldata_deberta_xlarge_forzen_22_sequential'].to_numpy()
r3 = 1 - results3['sequential_nogpt_deberta_xlarge'].to_numpy()

test = np.stack((r1, r2, r3), axis=0).mean(axis=0)
test.shape

(1500,)

In [91]:
r1_new = result.x[0] * r1
r2_new = result.x[1] * r2
r3_new = result.x[2] * r3

res = r1_new + r2_new + r3_new
res

array([0.15826581, 0.25350449, 0.04561327, ..., 0.06345979, 0.02640206,
       0.38677652])

In [95]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(path + '/lp_ensemble.csv', index=False)

In [97]:
res = pd.read_csv(path + '/lp_ensemble.csv')['p(Hallucination)'].to_numpy()
res

array([0.15826581, 0.25350449, 0.04561327, ..., 0.06345979, 0.02640206,
       0.38677652])

In [99]:
labels_sequential = np.where(r3 > 0.5, 1, 0)
labels_ensemble = np.where(res > 0.5, 1, 0)
differences = np.logical_xor(labels_sequential, labels_ensemble).sum()
differences

54